In [3]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

In [4]:
# make sure pandas is version 1.0 or higher
# make sure networkx is verion 2.4 or higher
print(pd.__version__)
print(nx.__version__)

2.0.3
3.1


In [5]:
from ema_workbench import (
    Model,
    Policy,
    ema_logging,
    SequentialEvaluator,
    MultiprocessingEvaluator,
)
from dike_model_function import DikeNetwork  # @UnresolvedImport
from problem_formulation import get_model_for_problem_formulation, sum_over, sum_over_time



In [17]:
ema_logging.log_to_stderr(ema_logging.INFO)

# choose problem formulation number, between 0-5
# each problem formulation has its own list of outcomes
dike_model, planning_steps = get_model_for_problem_formulation(2)

In [18]:
# enlisting uncertainties, their types (RealParameter/IntegerParameter/CategoricalParameter), lower boundary, and upper boundary
import copy

for unc in dike_model.uncertainties:
    print(repr(unc))

uncertainties = copy.deepcopy(dike_model.uncertainties)

CategoricalParameter('discount rate 0', [0, 1, 2, 3])
CategoricalParameter('discount rate 1', [0, 1, 2, 3])
CategoricalParameter('discount rate 2', [0, 1, 2, 3])
IntegerParameter('A.0_ID flood wave shape', 0, 132, resolution=None, default=None, variable_name=['A.0_ID flood wave shape'], pff=False)
RealParameter('A.1_Bmax', 30, 350, resolution=None, default=None, variable_name=['A.1_Bmax'], pff=False)
RealParameter('A.1_pfail', 0, 1, resolution=None, default=None, variable_name=['A.1_pfail'], pff=False)
CategoricalParameter('A.1_Brate', [0, 1, 2])
RealParameter('A.2_Bmax', 30, 350, resolution=None, default=None, variable_name=['A.2_Bmax'], pff=False)
RealParameter('A.2_pfail', 0, 1, resolution=None, default=None, variable_name=['A.2_pfail'], pff=False)
CategoricalParameter('A.2_Brate', [0, 1, 2])
RealParameter('A.3_Bmax', 30, 350, resolution=None, default=None, variable_name=['A.3_Bmax'], pff=False)
RealParameter('A.3_pfail', 0, 1, resolution=None, default=None, variable_name=['A.3_pfai

In [19]:
# enlisting policy levers, their types (RealParameter/IntegerParameter), lower boundary, and upper boundary
for policy in dike_model.levers:
    print(repr(policy))

levers = copy.deepcopy(dike_model.levers)

IntegerParameter('0_RfR 0', 0, 1, resolution=None, default=None, variable_name=['0_RfR 0'], pff=False)
IntegerParameter('0_RfR 1', 0, 1, resolution=None, default=None, variable_name=['0_RfR 1'], pff=False)
IntegerParameter('0_RfR 2', 0, 1, resolution=None, default=None, variable_name=['0_RfR 2'], pff=False)
IntegerParameter('1_RfR 0', 0, 1, resolution=None, default=None, variable_name=['1_RfR 0'], pff=False)
IntegerParameter('1_RfR 1', 0, 1, resolution=None, default=None, variable_name=['1_RfR 1'], pff=False)
IntegerParameter('1_RfR 2', 0, 1, resolution=None, default=None, variable_name=['1_RfR 2'], pff=False)
IntegerParameter('2_RfR 0', 0, 1, resolution=None, default=None, variable_name=['2_RfR 0'], pff=False)
IntegerParameter('2_RfR 1', 0, 1, resolution=None, default=None, variable_name=['2_RfR 1'], pff=False)
IntegerParameter('2_RfR 2', 0, 1, resolution=None, default=None, variable_name=['2_RfR 2'], pff=False)
IntegerParameter('3_RfR 0', 0, 1, resolution=None, default=None, variable

In [20]:
# enlisting outcomes
for outcome in dike_model.outcomes:
    print(repr(outcome))

ScalarOutcome('Expected Annual Damage', variable_name=('A.1_Expected Annual Damage', 'A.2_Expected Annual Damage', 'A.3_Expected Annual Damage', 'A.4_Expected Annual Damage', 'A.5_Expected Annual Damage'), function=<function sum_over at 0x000001CC6575BBA0>)
ScalarOutcome('Dike Investment Costs', variable_name=('A.1_Dike Investment Costs', 'A.2_Dike Investment Costs', 'A.3_Dike Investment Costs', 'A.4_Dike Investment Costs', 'A.5_Dike Investment Costs'), function=<function sum_over at 0x000001CC6575BBA0>)
ScalarOutcome('RfR Investment Costs', variable_name=('RfR Total Costs',), function=<function sum_over at 0x000001CC6575BBA0>)
ScalarOutcome('Evacuation Costs', variable_name=('Expected Evacuation Costs',), function=<function sum_over at 0x000001CC6575BBA0>)
ScalarOutcome('Expected Number of Deaths', variable_name=('A.1_Expected Number of Deaths', 'A.2_Expected Number of Deaths', 'A.3_Expected Number of Deaths', 'A.4_Expected Number of Deaths', 'A.5_Expected Number of Deaths'), function

In [21]:
# running the model through EMA workbench
with MultiprocessingEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(scenarios=50, policies=4)

[MainProcess/INFO] pool started with 12 workers
[MainProcess/INFO] performing 50 scenarios * 4 policies * 1 model(s) = 200 experiments
100%|████████████████████████████████████████| 200/200 [00:30<00:00,  6.47it/s]
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool


In [22]:
# observing the simulation runs
experiments, outcomes = results
print(outcomes.keys())
experiments

dict_keys(['Expected Annual Damage', 'Dike Investment Costs', 'RfR Investment Costs', 'Evacuation Costs', 'Expected Number of Deaths'])


,A.0_ID flood wave shape,A.1_Bmax,A.1_Brate,A.1_pfail,A.2_Bmax,A.2_Brate,A.2_pfail,A.3_Bmax,A.3_Brate,A.3_pfail,...,A.4_DikeIncrease 0,A.4_DikeIncrease 1,A.4_DikeIncrease 2,A.5_DikeIncrease 0,A.5_DikeIncrease 1,A.5_DikeIncrease 2,EWS_DaysToThreat,scenario,policy,model
0,0,247.205283,10.0,0.664077,236.676645,1.5,0.054812,68.898952,10.0,0.563560,...,3,3,0,2,0,2,0,158,154,dikesnet
1,60,60.338034,1.0,0.906600,135.016767,1.5,0.908551,165.546785,1.5,0.402633,...,3,3,0,2,0,2,0,159,154,dikesnet
2,114,255.233828,1.5,0.835157,126.643156,10.0,0.828499,342.750524,1.0,0.723729,...,3,3,0,2,0,2,0,160,154,dikesnet
3,98,275.368495,10.0,0.407247,313.126604,1.5,0.096659,187.831450,1.0,0.123011,...,3,3,0,2,0,2,0,161,154,dikesnet
4,41,78.228741,10.0,0.526537,187.790088,1.5,0.866286,172.465148,1.5,0.184826,...,3,3,0,2,0,2,0,162,154,dikesnet
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,62,238.176492,1.0,0.899904,245.517236,10.0,0.407298,75.456003,1.5,0.869392,...,2,1,2,3,4,2,3,203,157,dikesnet
196,91,280.166389,1.5,0.277761,74.745402,1.0,0.972846,181.756991,10.0,0.625035,...,2,1,2,3,4,2,3,204,157,dikesnet
197,56,312.405834,10.0,0.696594,145.351237,10.0,0.421926,325.211118,1.5,0.981577,...,2,1,2,3,4,2,3,205,157,dikesnet
198,8,190.281024,1.5,0.723454,266.906858,10.0,0.606508,65.541713,1.5,0.440267,...,2,1,2,3,4,2,3,206,157,dikesnet


In [23]:
# only works because we have scalar outcomes
pd.DataFrame(outcomes)

,Expected Annual Damage,Dike Investment Costs,RfR Investment Costs,Evacuation Costs,Expected Number of Deaths
0,3.138065e+08,6.055853e+08,1.482300e+09,0.000000,0.206278
1,8.458916e+06,6.055853e+08,1.482300e+09,0.000000,0.008393
2,1.744254e+06,6.055853e+08,1.482300e+09,0.000000,0.001680
3,1.816941e+08,6.055853e+08,1.482300e+09,0.000000,0.131056
4,2.437189e+07,6.055853e+08,1.482300e+09,0.000000,0.025289
...,...,...,...,...,...
195,3.596042e+06,5.466728e+08,1.099700e+09,272.804596,0.000490
196,2.429640e+06,5.466728e+08,1.099700e+09,161.146883,0.000150
197,5.955223e+06,5.466728e+08,1.099700e+09,384.352115,0.000321
198,5.538697e+07,5.466728e+08,1.099700e+09,4080.065146,0.003315


In [24]:
# defining specific policies
# for example, policy 1 is about extra protection in upper boundary
# policy 2 is about extra protection in lower boundary
# policy 3 is extra protection in random locations


def get_do_nothing_dict():
    return {l.name: 0 for l in dike_model.levers}


policies = [
    Policy(
        "policy 1",
        **dict(
            get_do_nothing_dict(),
            **{"0_RfR 0": 1, "0_RfR 1": 1, "0_RfR 2": 1, "A.1_DikeIncrease 0": 5}
        )
    ),
    Policy(
        "policy 2",
        **dict(
            get_do_nothing_dict(),
            **{"4_RfR 0": 1, "4_RfR 1": 1, "4_RfR 2": 1, "A.5_DikeIncrease 0": 5}
        )
    ),
    Policy(
        "policy 3",
        **dict(
            get_do_nothing_dict(),
            **{"1_RfR 0": 1, "2_RfR 1": 1, "3_RfR 2": 1, "A.3_DikeIncrease 0": 5}
        )
    ),
]

In [25]:
# pass the policies list to EMA workbench experiment runs
n_scenarios = 100
with MultiprocessingEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(n_scenarios, policies)

[MainProcess/INFO] pool started with 12 workers
[MainProcess/INFO] performing 100 scenarios * 3 policies * 1 model(s) = 300 experiments
100%|████████████████████████████████████████| 300/300 [00:47<00:00,  6.33it/s]
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool


In [26]:
experiments, outcomes = results

In [27]:
# only works because we have scalar outcomes
pd.DataFrame(outcomes)

,Expected Annual Damage,Dike Investment Costs,RfR Investment Costs,Evacuation Costs,Expected Number of Deaths
0,8.167305e+08,5.397251e+07,253800000.0,0.0,0.899376
1,3.221137e+08,5.397251e+07,253800000.0,0.0,0.261957
2,1.141255e+08,5.397251e+07,253800000.0,0.0,0.132941
3,4.011993e+08,5.397251e+07,253800000.0,0.0,0.399451
4,5.148275e+08,5.397251e+07,253800000.0,0.0,1.027389
...,...,...,...,...,...
295,1.027849e+08,2.879840e+07,369700000.0,0.0,0.104628
296,2.419468e+09,2.879840e+07,369700000.0,0.0,1.721550
297,3.855063e+08,2.879840e+07,369700000.0,0.0,0.497921
298,2.870363e+09,2.879840e+07,369700000.0,0.0,1.741875
